In [ ]:
import pandas as pd
import numpy as np
import math
from itertools import combinations
from collections import defaultdict
from sklearn import metrics, calibration
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm
from scipy import stats
from statsmodels.nonparametric.smoothers_lowess import lowess
import statsmodels.api as sm
from pycaleva import CalibrationEvaluator # fork of https://martinweigl.github.io/pycaleva/ at https://github.com/KonKob/pycaleva/
from nri_f import nri
from confidenceinterval import roc_auc_score

import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
from datetime import datetime
import re
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go

from ESCAPE_plotting import adjust_color_brightness, plot_dca
from ESCAPE_evaluation import nagelkerke_coxsnell, calculate_dca, categorical_nri, bootstrap_nri, calculate_nri_with_threshold, smooth_array, bootstrap_ci, calculate_discrimination_slope
from ESCAPE_stats import pairwise_comparison, chi2_or_fisher_test, mwu_or_ttest, build_flat_summary_dict, comparisons_versus_reference, build_flat_summary_reference_dict

## Loading and preprocessing of data

In [ ]:
# access file
data_file = "data.xlsx"

In [ ]:
# load table and transform
df_data = pd.read_excel(data_file, header=None, index_col=0)
df_data = df_data.T
# remove empty entries
df_data = df_data.loc[df_data["pseudonym"].notna(), :]
# replace 'error: missing input'
df_data = df_data.replace({'error: missing input': np.nan})
df_data.tail()

In [ ]:
# check for duplicates
df_data.loc[df_data.duplicated(subset=["pseudonym"]), "pseudonym"]

In [ ]:
print("pseudonyms to remove:", ", ".join(df_data.loc[df_data["exclusion due to exclusion criteria"]=="yes", "pseudonym"].astype(str).values))
df_data = df_data.loc[df_data["exclusion due to exclusion criteria"]!="yes", :]
print("pseudonyms with unfinished entry:", ", ".join(df_data.loc[df_data["entry done"]=="no", "pseudonym"].astype(str).values))
df_data = df_data.loc[df_data["entry done"]!="no", :]

In [ ]:
cohorts = df_data["cohort"].unique()

## Definitions relevant for analysis

In [ ]:
colors_hex = {"CardioExplorer": "#1F18A2", 'CardioExplorer individual PTP': "#1F18A2", "ESC 2024 PTP (formula)": "#AF1329", "ESC 2024 RF-CL (formula)": "#8C0D1F", "ESC 2019 PTP": '#C21830', "ESC 2013 PTP": "#F28C8C", "ESC 2024 CACS-CL (formula)": "#5A0A16", "ACC 2021 PTP": "#00294C", "ACC 2012 PTP": "#003366", 'Diamond-Forrester': "#08F767", "ESC 2024 RF-CL (formula with default 0 for number of risk factors and symptom score)": "#8C0D1F", "treat all": "#CECECE", "treat none": "#000000", "PROMISE Minimal-Risk Score": "#4c65a5", "CAD consortium score 2012 Basic PTP": "#d9dfee", "CAD consortium score 2012 Clinical PTP": "#d3d6e9", "clinical assessment of CAD pretest probability": "#FF9900"} 

In [ ]:
missing_values_of_interest = ['previous CAD diagnostic', 'CAD previously known', 'length of stay in ED (for ED cohort only)', 'performed, planned, or recommended initial diagnostics', 'inpatient admission', 'clinical assessment of CAD pretest probability', 'diagnosis at discharge or inpatient admission', 'age [years]', 'sex', 'height [cm]', 'weight [kg]', 'presence and type of chest pain (CE)', 'presence and type of chest pain (ESC 2013/ACC 2012)', 'thoracic pain (ESC 2019)', 'thoracic pain (ACC 2021)', 'nicotine consumption (CE)', 'nicotine consumption (ESC 2024)', 'family history', 'dyslipidemia', 'hypertension', 'diabetes', 'diabetes medication', 'cholesterol lowering medication', 'tc aggregation inhibitor', 'RAS inhibitor', 'ca antagonist', 'betablocker', 'diuretic', 'organic nitrate', 'systolic blood pressure [mmHg]', 'diastolic blood pressure [mmHg]', 'pathological Q-waves at ECG', 'pancreatic amylase [U/l]', 'alkaline phosphatase [U/l]', 'hs troponin T [pg/ml]', 'alanin-aminotransferase [U/l]', 'glucose [mg/dl]', 'bilirubin (total) [mg/dl]', 'urea [mg/dl]', 'uric acid [mg/dl]', 'cholesterol (total) [mg/dl]', 'high-density lipoprotein cholesterol [mg/dl]', 'low-density lipoprotein cholesterol [mg/dl]', 'protein (total) [g/l]', 'albumin [g/l]', 'leucocytes [*1000/yl]', 'mean corpuscular haemoglobin concentration [g/dl]', 'coronary artery calcification score from previous chest CT scans', 'CE (primary care calibration) score', 'CE (primary care calibration) PTP', 'Diamond-Forrester', 'ACC 2012 PTP', 'ESC 2013 PTP', 'ESC 2019 PTP', 'ACC 2021 PTP', 'PROMISE Minimal-Risk Score', 'ESC 2024 RF-CL (table)', 'ESC 2024 PTP (formula)', 'ESC 2024 RF-CL (formula)', 'ESC 2024 CACS-CL (formula)', "diabetes medication"]

In [ ]:
# outcome definitions
df_data["atrial fibrillation dummy outcome"] = df_data["atrial fibrillation"].replace({"yes": 1, "no": 0})
df_data["CAD progression or new CAD"] = df_data["new CAD or CAD progression 3 months"].replace({"inconclusive": np.nan, "excluded": 0, "diagnosed": 1})
df_data["known CAD or new CAD"] = ((df_data["new CAD or CAD progression 3 months"]=="diagnosed") | (df_data["CAD previously known"] == "yes")).astype(int)

outcomes = ["atrial fibrillation dummy outcome", "CAD progression or new CAD", "known CAD or new CAD"]

In [ ]:
scores_to_compare = [
    ["CardioExplorer individual PTP", "ESC 2024 PTP (formula)", "ESC 2024 RF-CL (formula with default 0 for number of risk factors and symptom score)", "clinical assessment of CAD pretest probability"],
    ["CardioExplorer", "ESC 2024 PTP (formula)", "ESC 2024 RF-CL (formula with default 0 for number of risk factors and symptom score)", "clinical assessment of CAD pretest probability"],
    ["CardioExplorer individual PTP", "ESC 2024 PTP (formula)", "ESC 2024 RF-CL (formula)", "clinical assessment of CAD pretest probability"],
                    ]

In [ ]:
pseudonyms_per_group = {
    "total": df_data["pseudonym"].unique(), 
    "ED": df_data.loc[df_data["cohort"]=="ED", "pseudonym"].unique(),
    "DZHI": df_data.loc[df_data["cohort"]=="DZHI", "pseudonym"].unique(),
    "ED_CCS": df_data.loc[(df_data["cohort"]=="ED") & (df_data["inclusion criterium"]=="CCS"), "pseudonym"].unique(),
    "ED_AF": df_data.loc[(df_data["cohort"]=="ED") & (df_data["inclusion criterium"]=="AF"), "pseudonym"].unique(),
    "DZHI_HCM": df_data.loc[(df_data["cohort"]=="DZHI") & (df_data["inclusion criterium"]=="HCM"), "pseudonym"].unique(),
    "DZHI_noHCM": df_data.loc[(df_data["cohort"]=="DZHI") & (df_data["inclusion criterium"]!="HCM"), "pseudonym"].unique(),
    "DZHI_nopriorCAD": df_data.loc[(df_data["cohort"]=="DZHI") & (df_data["CAD previously known"]=="no"), "pseudonym"].unique(),
    "DZHI_priorCAD": df_data.loc[(df_data["cohort"]=="DZHI") & (df_data["CAD previously known"]=="yes"), "pseudonym"].unique(),
    "allcohorts_nopriorCAD": df_data.loc[(df_data["CAD previously known"]=="no"), "pseudonym"].unique(),
    "allcohorts_priorCAD": df_data.loc[(df_data["CAD previously known"]=="yes"), "pseudonym"].unique(),
    "male": df_data.loc[(df_data["sex"]=="male"), "pseudonym"].unique(),
    "female": df_data.loc[(df_data["sex"]=="female"), "pseudonym"].unique(),
    "no_stdtroponin_ED": df_data.loc[(df_data["hs troponin T [pg/ml]"].notna()) & (df_data["cohort"]=="ED"), "pseudonym"].unique(),
    "stdtroponin_ED": df_data.loc[(~df_data["hs troponin T [pg/ml]"].notna()) & (df_data["cohort"]=="ED"), "pseudonym"].unique(),
    "furtherCADdiagnostic": df_data.loc[df_data["CAD progression or new CAD"].notna(), "pseudonym"].unique(),
    "no_furtherCADdiagnostic": df_data.loc[~df_data["CAD progression or new CAD"].notna(), "pseudonym"].unique(),
    "primary_diagnosed": df_data.loc[df_data["CAD progression or new CAD"]==1, "pseudonym"].unique(),
    "primary_excluded": df_data.loc[df_data["CAD progression or new CAD"]==0, "pseudonym"].unique(),
                        }

per_group_variables_scale = {
    
    "diabetes": "categorial",
    "diabetes medication": "categorial",
    "dyslipidemia": "categorial",
    "dyspnea": "categorial",
    "family history": "categorial",
    "female sex": "categorial",
    "hypertension": "categorial",
    "inpatient admission": "categorial",
    "pathological Q-waves at ECG": "categorial",
    "atrial fibrillation": "categorial",
    "CAD previously known": "categorial",

    "presence and type of chest pain (ESC 2013/ACC 2012)": "categorial",
    "nicotine consumption (CE)": "categorial",
    "clinical assessment of CAD pretest probability": "categorial",
    "pathological Q-waves at ECG": "categorial", 
    "inpatient admission": "categorial", 
    "cholesterol lowering medication": "categorial", 
    "tc aggregation inhibitor": "categorial", 
    "RAS inhibitor": "categorial", 
    "ca antagonist": "categorial", 
    "betablocker": "categorial", 
    "diuretic": "categorial", 
    "organic nitrate": "categorial",

    "age [years]": "rational",
    "height [cm]": "rational",
    "weight [kg]": "rational",
    "coronary artery calcification score from previous chest CT scans": "rational",
    "systolic blood pressure [mmHg]": "rational",
    "diastolic blood pressure [mmHg]": "rational",
    "BMI [kg/m^2]": "rational", 
    "pancreatic amylase [U/l]": "rational", 
    "alkaline phosphatase [U/l]": "rational", 
    "hs troponin T [pg/ml]": "rational",
    "alanin-aminotransferase [U/l]": "rational", 
    "glucose [mg/dl]": "rational", 
    "bilirubin (total) [mg/dl]": "rational", 
    "urea [mg/dl]": "rational", 
    "uric acid [mg/dl]": "rational", 
    "cholesterol (total) [mg/dl]": "rational", 
    "high-density lipoprotein cholesterol [mg/dl]": "rational", 
    "low-density lipoprotein cholesterol [mg/dl]": "rational", 
    "protein (total) [g/l]": "rational", 
    "albumin [g/l]": "rational", 
    "leucocytes [*1000/yl]": "rational", 
    "mean corpuscular haemoglobin concentration [g/dl]": "rational",

    "previous CAD diagnostic": "free text",
    "performed, planned, or recommended initial diagnostics": "free text",
    "diagnosis at discharge or inpatient admission": "free text",
    "diagnosis at discharge or inpatient admission": "free text",
                                
                           }
for outcome in outcomes:
    per_group_variables_scale[outcome] = "categorial"

## Missing data

In [ ]:
df_nan_replaced = df_data.replace("error: missing input", np.nan)
df_number_of_missing_values = df_nan_replaced.isna().sum()[missing_values_of_interest]
df_missing_values = df_number_of_missing_values.loc[df_number_of_missing_values>0]
df_missing_values_percent = df_missing_values/df_data.shape[0] 
df_values_present_percent = 1 - df_missing_values_percent
missing_parameter_categories = [("anamnesis", ['presence and type of chest pain (ESC 2013/ACC 2012)', 'thoracic pain (ESC 2019)', 'thoracic pain (ACC 2021)', 'nicotine consumption (CE)', 'nicotine consumption (ESC 2024)',
       'family history', 'dyslipidemia', 'hypertension', 'diabetes', 'cholesterol lowering medication', 'tc aggregation inhibitor', 'RAS inhibitor', 'ca antagonist', 'betablocker', 'diuretic', 'organic nitrate', "diabetes medication"]),
                     ("laboratory results", ['pancreatic amylase [U/l]', 'hs troponin T [pg/ml]', 'bilirubin (total) [mg/dl]', 'cholesterol (total) [mg/dl]', 'high-density lipoprotein cholesterol [mg/dl]',
       'low-density lipoprotein cholesterol [mg/dl]', 'albumin [g/l]', 'alkaline phosphatase [U/l]', 'alanin-aminotransferase [U/l]', 'uric acid [mg/dl]', 'protein (total) [g/l]']),
                    ("test results", ['coronary artery calcification score from previous chest CT scans', 'systolic blood pressure [mmHg]', 'diastolic blood pressure [mmHg]', 'pathological Q-waves at ECG', ]),
                    ("scores", ['CE (primary care calibration) score', 'Diamond-Forrester', 'ACC 2012 PTP', 'ESC 2013 PTP', 'ESC 2019 PTP', 'ACC 2021 PTP', 'PROMISE Minimal-Risk Score', 'ESC 2024 RF-CL (table)',
       'ESC 2024 RF-CL (formula)', 'ESC 2024 CACS-CL (formula)']),
                    ("data extraction", ['diagnosis at discharge or inpatient admission', 'previous CAD diagnostic']) ,                         
                    ("outcomes", outcomes)
                               ]
missing_parameter_categories_dict = dict(missing_parameter_categories)
defined_missing_parameters = [p for _, params in missing_parameter_categories for p in params]
undefined_missing_parameters = [p for p in df_missing_values.index if p not in defined_missing_parameters]
if undefined_missing_parameters:
    print("The following parameters are missing but do not belong to a category!", undefined_missing_parameters)
print("The following parameters are missing for some entries.")
for category, params in missing_parameter_categories:
    print(category)
    params_really_missing = [p for p in params if p in df_missing_values.index]
    for p in params_really_missing:
        print("  - ", p, round(df_missing_values_percent[p], 2))

missing_parameter_categories_dict["laboratory results"].remove("hs troponin T [pg/ml]")
entries_with_missing_laboratory_results_except_trop = df_data.loc[(df_data[missing_parameter_categories_dict["laboratory results"]].isna().any(axis=1)), ["pseudonym", "date of inclusion", "inclusion criterium", "cohort"] + missing_parameter_categories_dict["laboratory results"]]
print("\n\nLaboratory results are missing in the following cohorts, inclusion criteria and subjects:")
for cohort in cohorts:
    print("\n", cohort)
    cohort_entries_with_missing_lab_results = entries_with_missing_laboratory_results_except_trop.loc[entries_with_missing_laboratory_results_except_trop["cohort"]==cohort]
    for inclusion_criterium in cohort_entries_with_missing_lab_results["inclusion criterium"].unique():
        print("  ", inclusion_criterium)
        print(cohort_entries_with_missing_lab_results.loc[cohort_entries_with_missing_lab_results["inclusion criterium"]==inclusion_criterium, missing_parameter_categories_dict["laboratory results"]].isna().sum(), cohort_entries_with_missing_lab_results.loc[cohort_entries_with_missing_lab_results["inclusion criterium"]==inclusion_criterium, "pseudonym"].values)

In [ ]:
df_data["hs troponin T [pg/ml]"] = df_data["hs troponin T [pg/ml]"].replace({"<3": 0})
df_data["alanin-aminotransferase [U/l]"] = df_data["alanin-aminotransferase [U/l]"].replace({"<5": 0})
df_data["bilirubin (total) [mg/dl]"] = df_data["bilirubin (total) [mg/dl]"].replace({"<0.2": 0})

## variables per cohort 

In [ ]:
group_df_list_for_binaries_or_continous = []
group_df_list_for_categorial = []
group_variable_comparison = pd.DataFrame({}, columns=["variable", "mean", "std", "group", "n", "n information available", "group n"])

for group, pseudonyms in pseudonyms_per_group.items():
    binary_variables = ["CAD previously known", "family history", "dyslipidemia", "hypertension", "diabetes", "atrial fibrillation", "pathological Q-waves at ECG", "inpatient admission", "cholesterol lowering medication", "tc aggregation inhibitor", "RAS inhibitor", "ca antagonist", "betablocker", "diuretic", "organic nitrate"]
    for outcome in outcomes:
        binary_variables.append(outcome)
    numerical_variables = ["age [years]", "height [cm]", "weight [kg]", "BMI [kg/m^2]", "systolic blood pressure [mmHg]", "diastolic blood pressure [mmHg]", "coronary artery calcification score from previous chest CT scans", "pancreatic amylase [U/l]", "alkaline phosphatase [U/l]", "hs troponin T [pg/ml]", "alanin-aminotransferase [U/l]", "glucose [mg/dl]", "bilirubin (total) [mg/dl]", "urea [mg/dl]", "uric acid [mg/dl]", "cholesterol (total) [mg/dl]", "high-density lipoprotein cholesterol [mg/dl]", "low-density lipoprotein cholesterol [mg/dl]", "protein (total) [g/l]", "albumin [g/l]", "leucocytes [*1000/yl]", "mean corpuscular haemoglobin concentration [g/dl]"]
    group_df = df_data.loc[df_data["pseudonym"].isin(pseudonyms), :]
    # binary variables
    group_df["female sex"] = group_df["sex"]=="female"
    group_df["dyspnea"] = group_df["dyspnoea as shortness of breath and/or trouble catching breath aggravated by physical exertion (ESC 2024)"]=="yes"
    for v in binary_variables:
        group_df[v] = group_df[v].replace({"yes": 1, "M61": 1, "discharge against medical advice": 1, "no": 0}).astype(float)
    group_df["group"] = group
    missing_values = {}
    for v in numerical_variables + binary_variables + ["sex", "dyspnoea as shortness of breath and/or trouble catching breath aggravated by physical exertion (ESC 2024)", "nicotine consumption (CE)", "clinical assessment of CAD pretest probability", "diabetes medication"]:
        if v=="dyspnoea as shortness of breath and/or trouble catching breath aggravated by physical exertion (ESC 2024)":
            key = "dyspnea"
        elif v=="sex":
            key = "female sex"
        else:
            key = v
        missing_values[key] = group_df.loc[group_df[key].isna(), :].shape[0] 
        
    chest_pain = group_df["presence and type of chest pain (ESC 2013/ACC 2012)"].value_counts()
    chest_pain["none"] = group_df.loc[group_df["presence and type of chest pain (ESC 2013/ACC 2012)"].isna(), :].shape[0]
    missing_values["chest pain"] = 0
    nicotine_consumption = group_df["nicotine consumption (CE)"].value_counts()
    clinical_cad_ptp = group_df["clinical assessment of CAD pretest probability"].value_counts()
    diabetes_medication = group_df["diabetes medication"].value_counts()

    group_df_list_for_binaries_or_continous.append(group_df[numerical_variables + binary_variables + ["female sex", "dyspnea", "group"]])
    group_df_list_for_categorial.append(group_df[["presence and type of chest pain (ESC 2013/ACC 2012)", "nicotine consumption (CE)", "clinical assessment of CAD pretest probability", "diabetes medication", "group"]])
    
    n = group_df.shape[0]
    means = group_df.loc[:, numerical_variables + binary_variables + ["female sex", "dyspnea"]].mean(numeric_only=True) # continuous/binary categories
    stds = group_df.loc[:, numerical_variables].std(numeric_only=True) # continuous categories
    positive_values = group_df.loc[:, binary_variables + ["female sex", "dyspnea"]].sum(numeric_only=True)
    for v in means.index:
        group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[v, round(means[v], 2), round(stds[v], 2) if v in stds else np.nan, group, int(positive_values[v]) if v not in stds else np.nan, n-missing_values[v], n]], columns=group_variable_comparison.columns)], ignore_index=True)
    for v in df_data["nicotine consumption (CE)"].unique():
        if not pd.isna(v):
            if v in nicotine_consumption:
                group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("nicotine consumption (CE)", v), np.nan, np.nan, group, nicotine_consumption[v], n-missing_values["nicotine consumption (CE)"], n]], columns=group_variable_comparison.columns)], ignore_index=True)
            else:
                group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("nicotine consumption (CE)", v), np.nan, np.nan, group, 0, n-missing_values["nicotine consumption (CE)"], n]], columns=group_variable_comparison.columns)], ignore_index=True)

    for v in df_data["diabetes medication"].unique():
        if not pd.isna(v):
            if v in diabetes_medication:
                group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("diabetes medication", v), np.nan, np.nan, group, diabetes_medication[v], n-missing_values["diabetes medication"], n]], columns=group_variable_comparison.columns)], ignore_index=True)
            else:
                group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("diabetes medication", v), np.nan, np.nan, group, 0, n-missing_values["diabetes medication"], n]], columns=group_variable_comparison.columns)], ignore_index=True)
                
    for v in df_data["presence and type of chest pain (ESC 2013/ACC 2012)"].unique():
        if not pd.isna(v):
            if v in chest_pain:
                group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("presence and type of chest pain (ESC 2013/ACC 2012)", v), np.nan, np.nan, group, chest_pain[v], n-missing_values["chest pain"], n]], columns=group_variable_comparison.columns)], ignore_index=True)
            else:
                group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("presence and type of chest pain (ESC 2013/ACC 2012)", v), np.nan, np.nan, group, 0, n-missing_values["chest pain"], n]], columns=group_variable_comparison.columns)], ignore_index=True)
    group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("presence and type of chest pain (ESC 2013/ACC 2012)",  "none"), np.nan, np.nan, group, chest_pain["none"], n-missing_values["chest pain"], n]], columns=group_variable_comparison.columns)], ignore_index=True)
    for v in df_data["clinical assessment of CAD pretest probability"].unique():
        if not pd.isna(v):
            if v in clinical_cad_ptp:
                    group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("clinical assessment of CAD pretest probability", v), np.nan, np.nan, group, clinical_cad_ptp[v], n-missing_values["clinical assessment of CAD pretest probability"], n]], columns=group_variable_comparison.columns)], ignore_index=True)
            else:
                group_variable_comparison = pd.concat([group_variable_comparison, pd.DataFrame([[("clinical assessment of CAD pretest probability", v), np.nan, np.nan, group, 0, n-missing_values["clinical assessment of CAD pretest probability"], n]], columns=group_variable_comparison.columns)], ignore_index=True)

pd.options.mode.chained_assignment = "warn"

group_comparison_subject_binaries_or_continous = pd.concat(group_df_list_for_binaries_or_continous, ignore_index=True).replace({False: 0, True: 1})
group_comparison_subject_categorial = pd.concat(group_df_list_for_categorial, ignore_index=True)
group_comparison_subject_categorial["presence and type of chest pain (ESC 2013/ACC 2012)"]=group_comparison_subject_categorial["presence and type of chest pain (ESC 2013/ACC 2012)"].replace(np.nan, "None")

group_comparison_per_subject = pd.merge(group_comparison_subject_categorial, group_comparison_subject_binaries_or_continous, left_index=True, right_index=True)
group_comparison_per_subject = group_comparison_per_subject.rename(columns={"group_x": "group"})
group_comparison_per_subject = group_comparison_per_subject.drop("group_y", axis=1)

per_group_variables_statistics = pairwise_comparison(group_comparison_per_subject, per_group_variables_scale, group_pairs = [("ED", "DZHI"), ("ED_CCS", "ED_AF"), ("DZHI_HCM", "DZHI_noHCM"), ("DZHI_priorCAD", "DZHI_nopriorCAD"), ("allcohorts_nopriorCAD", "allcohorts_priorCAD"), ("male", "female"), ("no_stdtroponin_noHCM", "stdtroponin_noHCM"), ("no_stdtroponin_ED", "stdtroponin_ED")])
flat_summary_dict = build_flat_summary_dict(group_variable_comparison, per_group_variables_statistics, per_group_variables_scale)
per_group_variables = pd.DataFrame.from_dict(flat_summary_dict, orient="index")

In [ ]:
binary_and_continuous_reference = pd.read_excel("e.xlsx", index_col=0, sheet_name="binary_and_continuous_variables")
binary_and_continuous_reference["mean_standard_unit"] = binary_and_continuous_reference["mean_standard_unit"].astype(float)
categorial_reference = pd.read_excel("e.xlsx", index_col=[0, 1], sheet_name="categorial_variables")
reference = {"binary_and_continuous": binary_and_continuous_reference, "categorial": categorial_reference}

In [ ]:
variables_against_reference = comparisons_versus_reference(df_data, reference, pseudonyms_per_group, total_n=696, name = "e")
summary_reference_dict = build_flat_summary_reference_dict(variables_against_reference, name = "e")

In [ ]:
against_reference_variables = pd.DataFrame.from_dict(summary_reference_dict, orient="index")

In [ ]:
binary_and_continuous_reference_z = pd.read_excel("z.xlsx", index_col=0, sheet_name="binary_and_continuous_variables")
binary_and_continuous_reference_z["mean_standard_unit"] = binary_and_continuous_reference_z["mean_standard_unit"].astype(float)
categorial_reference_z = pd.read_excel("z.xlsx", index_col=[0, 1], sheet_name="categorial_variables")
reference_z = {"binary_and_continuous": binary_and_continuous_reference_z, "categorial": categorial_reference_z}

In [ ]:
variables_against_reference_z = comparisons_versus_reference(df_data, reference_z, pseudonyms_per_group, total_n=987, name="z")
summary_reference_dict_z = build_flat_summary_reference_dict(variables_against_reference_z, name = "z")

In [ ]:
against_reference_variables_z = pd.DataFrame.from_dict(summary_reference_dict_z, orient="index")

In [ ]:
print("Time in ED per group")
for group in ["ED", "ED_CCS", "ED_AF"]: #"ED_inpatient", "ED_no_inpatient", "ED_inpatient_CCS", "ED_no_inpatient_CCS", "ED_inpatient_AF", "ED_no_inpatient_AF"
    print(group, round(df_data.loc[df_data["pseudonym"].isin(pseudonyms_per_group[group]), "length of stay in ED (for ED cohort only)"].apply(lambda x: x.total_seconds()/3600).mean(), 2), "+-", round(df_data.loc[df_data["pseudonym"].isin(pseudonyms_per_group[group]), "length of stay in ED (for ED cohort only)"].apply(lambda x: x.total_seconds()/3600).std(), 2), "mean +- std")
    print(group, round(df_data.loc[df_data["pseudonym"].isin(pseudonyms_per_group[group]), "length of stay in ED (for ED cohort only)"].apply(lambda x: x.total_seconds()/3600).median(), 2), "(", round(df_data.loc[df_data["pseudonym"].isin(pseudonyms_per_group[group]), "length of stay in ED (for ED cohort only)"].apply(lambda x: x.total_seconds()/3600).quantile(0.25), 2), "-", round(df_data.loc[df_data["pseudonym"].isin(pseudonyms_per_group[group]), "length of stay in ED (for ED cohort only)"].apply(lambda x: x.total_seconds()/3600).quantile(0.75), 2), ")", "median (25-75% quantile range)")
    print(group, "min:", round(df_data.loc[df_data["pseudonym"].isin(pseudonyms_per_group[group]), "length of stay in ED (for ED cohort only)"].apply(lambda x: x.total_seconds()/3600).min(), 2), "max:", round(df_data.loc[df_data["pseudonym"].isin(pseudonyms_per_group[group]), "length of stay in ED (for ED cohort only)"].apply(lambda x: x.total_seconds()/3600).max(), 2))

## slice for a specific cohort

In [ ]:
# slice for specific cohort
cohort_slice = "total"    # ED, DZHI, male, female, HCM, DZHI_HCM, DZHI_noHCM, ED_CCS, stdtroponin, no_stdtroponin
df_data = df_data.loc[df_data["pseudonym"].isin(pseudonyms_per_group[cohort_slice]), :]

## evaluation of calibration

In [ ]:
CE_PTP_categories = {"<5%": 0.05, "~20%": 0.2, "~50%": 0.5, ">75%": 0.75, "error:missing input": np.nan} 
clinical_PTP_categories = {"very low": 0.05, "low": 0.45, "moderate": 0.45, "high": 0.45, "very high": 0.9999}
PROMISE_PTP_categories = lambda l: 0.05 if l > 0.34 else 0.9999
df_scores = df_data.loc[:, ["pseudonym", "CE (primary care calibration) score", "ESC 2024 PTP (formula)", "ESC 2024 RF-CL (formula)", "Diamond-Forrester", "ACC 2012 PTP", "ESC 2013 PTP", "ESC 2019 PTP", "ACC 2021 PTP", "ESC 2024 RF-CL (formula with default 0 for number of risk factors and symptom score)", "PROMISE Minimal-Risk Score", "CAD consortium score 2012 Basic PTP", "CAD consortium score 2012 Clinical PTP", "clinical assessment of CAD pretest probability", "CE individual PTP"]]
df_scores.loc[:, ["Diamond-Forrester", "ACC 2012 PTP", "ESC 2013 PTP", "ESC 2019 PTP", "ACC 2021 PTP"]] = df_scores.loc[:, ["Diamond-Forrester", "ACC 2012 PTP", "ESC 2013 PTP", "ESC 2019 PTP", "ACC 2021 PTP"]]/100
df_scores_copy = df_scores.copy()
df_scores.columns = pd.MultiIndex.from_product([df_scores.columns, ['score_values']])
df_scores_copy.columns = pd.MultiIndex.from_product([df_scores_copy.columns, ['ptp_values']])
df_scores = pd.concat([df_scores, df_scores_copy], axis=1)
df_scores = df_scores.sort_index(axis=1, level=0)

df_scores[("CE (primary care calibration) score", "ptp_values")] = df_data.loc[:, "CE (primary care calibration) PTP"].replace(CE_PTP_categories)
df_scores[("CE individual PTP", "score_values")] =  df_data.loc[:, "CE (primary care calibration) score"]
df_scores[("PROMISE Minimal-Risk Score", "ptp_values")] = df_data.loc[:, "PROMISE Minimal-Risk Score"].apply(PROMISE_PTP_categories) # PMRS provides only a binary classification based on a threshold of 0.34
df_scores[("clinical assessment of CAD pretest probability", "score_values")] = df_scores[("clinical assessment of CAD pretest probability", "score_values")].replace(clinical_PTP_categories)
df_scores[("clinical assessment of CAD pretest probability", "ptp_values")] = df_scores[("clinical assessment of CAD pretest probability", "score_values")].replace(clinical_PTP_categories)
df_scores = df_scores.drop(("pseudonym", "ptp_values"), axis=1)
df_scores.columns = pd.MultiIndex.from_tuples([("pseudonym", " ") if col == ("pseudonym", "score_values") else col for col in df_scores.columns])
df_scores.columns = pd.MultiIndex.from_tuples([("CardioExplorer", "score_values") if col == ("CE (primary care calibration) score", "score_values") else col for col in df_scores.columns])
df_scores.columns = pd.MultiIndex.from_tuples([("CardioExplorer", "ptp_values") if col == ("CE (primary care calibration) score", "ptp_values") else col for col in df_scores.columns])

df_scores.columns = pd.MultiIndex.from_tuples([("CardioExplorer individual PTP", "score_values") if col == ("CE individual PTP", "score_values") else col for col in df_scores.columns])
df_scores.columns = pd.MultiIndex.from_tuples([("CardioExplorer individual PTP", "ptp_values") if col == ("CE individual PTP", "ptp_values") else col for col in df_scores.columns])

In [ ]:
score_comparison_scores_and_outcomes = {}
for score_comparison in scores_to_compare:
    for score in score_comparison:
        print(score, df_scores.loc[:, score].dropna().shape[0])
    df_scores_for_comparison = df_scores.loc[:, [(score, v) for score in score_comparison for v in ["ptp_values", "score_values"]]+[("pseudonym", " ")]]
    df_scores_for_comparison = df_scores_for_comparison.dropna()
    df_scores_for_comparison = df_scores_for_comparison.convert_dtypes()
    df_scores_for_comparison["pseudonym"] = df_scores_for_comparison["pseudonym"].astype(int)
    score_comparison_scores_and_outcomes[" vs. ".join(score_comparison)] = {"score_comparison": score_comparison, "df_scores_for_comparison": df_scores_for_comparison}
    
for score_comparison_key in score_comparison_scores_and_outcomes:
    df_scores_for_comparison = score_comparison_scores_and_outcomes[score_comparison_key]["df_scores_for_comparison"]
    score_comparison = score_comparison_scores_and_outcomes[score_comparison_key]["score_comparison"]
    score_comparison_scores_and_outcomes[score_comparison_key]["outcomes"] = {}
    for outcome in outcomes:
        subjects_with_available_score = df_scores_for_comparison.loc[:, ("pseudonym", " ")].values
        subjects_with_available_outcomes = df_data.loc[df_data.loc[:, outcome].notna(), "pseudonym"].values
        subjects_with_available_score_and_outcome = [s for s in subjects_with_available_score if s in subjects_with_available_outcomes]
        print(score_comparison_key, "for", outcome, "\nsubjects with scores available:", len(subjects_with_available_score), "\nsubjects with outcome available:", len(subjects_with_available_outcomes), "\nsubjects with score and outcome available:", len(subjects_with_available_score_and_outcome))
        y_true = df_data.loc[df_data.loc[:, "pseudonym"].isin(subjects_with_available_score_and_outcome), outcome].replace({"yes": 1, "no": 0}).astype(int).values
        
        scores = {}
        for score in score_comparison:
            scores[score] = {"score_values": df_scores_for_comparison.loc[df_scores_for_comparison.loc[:, ("pseudonym", " ")].isin(subjects_with_available_score_and_outcome), (score, "score_values")].values, "ptp_values": df_scores_for_comparison.loc[df_scores_for_comparison.loc[:, ("pseudonym", " ")].isin(subjects_with_available_score_and_outcome), (score, "ptp_values")].values}
        score_comparison_scores_and_outcomes[score_comparison_key]["outcomes"][outcome] = {"y_true": y_true, "scores": scores}

In [ ]:
for score_comparison_key in score_comparison_scores_and_outcomes:
    # evaluation
    evaluation_per_score_and_outcome = {}
    for outcome, outcome_dict in score_comparison_scores_and_outcomes[score_comparison_key]["outcomes"].items():
        y_true = outcome_dict["y_true"]
        scores = outcome_dict["scores"]
        if len(np.unique(y_true)) == 2:
            prevalence = np.mean(y_true)
            n = len(y_true)
            evaluation_per_score_and_outcome[outcome] = {"extreme_strategies": {}, "scores": {}}
            dca_df_all = calculate_dca(y_true, np.ones(len(y_true)), model_name="treat all", harm=0)
            dca_df_none = calculate_dca(y_true, np.zeros(len(y_true)), model_name="treat none", harm=0)
            evaluation_per_score_and_outcome[outcome]["extreme_strategies"]["treat all"] = {"dca_df": dca_df_all}
            evaluation_per_score_and_outcome[outcome]["extreme_strategies"]["treat none"] = {"dca_df": dca_df_none}
            
            for score, score_dict in scores.items(): 
                y_pred_prob = score_dict["ptp_values"]
                score_values = score_dict["score_values"]
                evaluation_per_score_and_outcome[outcome]["scores"][score] = {}
                
                # overall performance
                evaluation_per_score_and_outcome[outcome]["scores"][score]["brier"] = metrics.brier_score_loss(y_true, y_pred_prob)
                evaluation_per_score_and_outcome[outcome]["scores"][score]["brier ci"] = bootstrap_ci(y_true, y_pred_prob, metrics.brier_score_loss)

                cs_r2, nk_r2, _, _ = nagelkerke_coxsnell(y_true, y_pred_prob)
                evaluation_per_score_and_outcome[outcome]["scores"][score]["nk_r2"] = nk_r2
                evaluation_per_score_and_outcome[outcome]["scores"][score]["cs_r2"] = cs_r2
                
                # discrimination
                auc, auc_ci = roc_auc_score(y_true, score_values, confidence_level=0.95)
                discrimination_slope = calculate_discrimination_slope(y_true, score_values)
                discrimination_slope_ci = bootstrap_ci(y_true, score_values, calculate_discrimination_slope)
        
                for threshold in [0.05, 0.15, 0.85]:
                    y_pred_outcome = y_pred_prob > threshold
                    tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred_outcome).ravel()
                    sensitivity = tp / (tp + fn)
                    specificity = tn / (tn + fp)
                    if (tp+fp) != 0:
                        ppv = tp / (tp + fp)
                    else:
                        ppv = np.nan
                    if (tn +fn) != 0:
                        npv = tn / (tn + fn)
                    else:
                        npv = np.nan
                    evaluation_per_score_and_outcome[outcome]["scores"][score][f"sensitivity at {threshold: .0%} threshold"] = sensitivity
                    evaluation_per_score_and_outcome[outcome]["scores"][score][f"specificity at {threshold: .0%} threshold"] = specificity
                    evaluation_per_score_and_outcome[outcome]["scores"][score][f"ppv at {threshold: .0%} threshold"] = ppv
                    evaluation_per_score_and_outcome[outcome]["scores"][score][f"npv at {threshold: .0%} threshold"] = npv
        
                evaluation_per_score_and_outcome[outcome]["scores"][score]["auc"] = auc
                evaluation_per_score_and_outcome[outcome]["scores"][score]["auc ci"] = auc_ci
                evaluation_per_score_and_outcome[outcome]["scores"][score]["discrimination slope"] = discrimination_slope
                evaluation_per_score_and_outcome[outcome]["scores"][score]["discrimination slope ci"] = discrimination_slope_ci
        
                # calibration
                y_true_np = y_true.astype(int)
                y_pred_prob_np = y_pred_prob.astype(float)
                y_pred_prob_np = np.clip(y_pred_prob.astype(float), 1e-15, 1 - 1e-15)
                logit_preds = np.log(y_pred_prob_np / (1 - y_pred_prob_np))
                X = sm.add_constant(logit_preds)

                calibration_intercept = "-"
                calibration_slope = "-"
                calibration_intercept_ci = ("-", "-")
                calibration_slope_ci = ("-", "-")
                
                if not len(np.unique(X)) <= 1:
                    try:
                        calib_model = sm.Logit(y_true_np, X).fit(disp=0)
                        calibration_intercept = calib_model.params[0]
                        calibration_slope = calib_model.params[1]
                        
                        ci = calib_model.conf_int()
                        calibration_intercept_ci = ci[0]
                        calibration_slope_ci = ci[1]
                    except np.linalg.LinAlgError:
                        pass
                
                observed, expected = np.sum(y_true), np.sum (y_pred_prob)
                oe_ratio = observed / expected if expected > 0 else np.nan
                obs_ci_lower, obs_ci_upper = (0, -np.log(0.05/2)) if observed == 0 else (stats.chi2.ppf(0.05/2, 2*observed) / 2, stats.chi2.ppf(1-0.05/2, 2*(observed+1)) / 2) # assuming Poisson distribution
                oe_ratio_ci_lower, oe_ratio_ci_upper = (obs_ci_lower / expected, obs_ci_upper / expected) if expected > 0 else (np.nan, np.nan)
                oe_ratio_pval = 2*min(stats.poisson.cdf(observed, expected), 1 - stats.poisson.cdf(observed - 1, expected))

                sort_idx = np.argsort(y_pred_prob_np)
                y_sorted, p_sorted = y_true_np[sort_idx], y_pred_prob_np[sort_idx]
                lowess_result = lowess(y_sorted, p_sorted, frac=0.2, return_sorted=False)
                calibrated_probs = np.empty_like(lowess_result)
                calibrated_probs[sort_idx] = lowess_result
                abs_diff = np.abs(calibrated_probs - y_pred_prob_np)
                ici, e50, e90, emax = np.mean(abs_diff), np.median(abs_diff), np.percentile(abs_diff, 90), np.max(abs_diff)
                
                if all([np.count_nonzero(y_true == y) > 1 for y in np.unique(y_true)]):
                    ce = CalibrationEvaluator(y_true, y_pred_prob, outsample=True, n_groups="auto")
                    ce_metrics_dict = ce.metrics()
                    hltest_result = ce.hosmerlemeshow(verbose=False)
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["hosmerlemeshow_statistic"] = hltest_result[0]
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["hosmerlemeshow_p"] = hltest_result[1]
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["ce"] = ce
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["ce_contingency_table"] = ce.contingency_table
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["adaptive calibration error"] = ce_metrics_dict[2]
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["maximum calibration error"] = ce_metrics_dict[3]
                else:
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["hosmerlemeshow_statistic"] = np.nan
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["hosmerlemeshow_p"] = np.nan
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["ce"] = np.nan
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["ce_contingency_table"] = np.nan
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["adaptive calibration error"] = np.nan
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["maximum calibration error"] = np.nan
                bins = [0.0, 0.05, 0.15, 0.5, 0.85, 1.0]
                guideline_thresholds = ["<5%", '5–15%', '15–50%', '50–85%', '>85%']
                guideline_threshold_labels = ["Very Low", 'Low', 'Moderate', 'High', "Very High"]
                df_preds = pd.DataFrame({'y_pred_prob': y_pred_prob, 'y_true': y_true, 'guideline_category': pd.cut(y_pred_prob, bins=bins, labels=guideline_thresholds, include_lowest=True)})
                guideline_category_prevalence_df = df_preds.groupby('guideline_category', observed=False)['y_true'].mean() # prevalence per group
                guideline_category_prevalence_df = guideline_category_prevalence_df.fillna("-")
                guideline_category_proportion_df = df_preds.groupby('guideline_category', observed=False).count()['y_true']/df_preds.shape[0] # proportion per group
                guideline_category_proportion_df = guideline_category_proportion_df.fillna(0)
        
                score_dict["logit_preds"] = logit_preds
                evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_intercept"] = calibration_intercept
                evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_intercept_ci"] = calibration_intercept_ci
                evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_slope"] = calibration_slope
                evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_slope_ci"] = calibration_slope_ci
                evaluation_per_score_and_outcome[outcome]["scores"][score]["expected vs. observed outcome ESC 2024 guideline categories"] = guideline_category_prevalence_df
                evaluation_per_score_and_outcome[outcome]["scores"][score]["proportion per ESC 2024 guideline categories"] = guideline_category_proportion_df
                evaluation_per_score_and_outcome[outcome]["scores"][score]["o:e ratio"] = oe_ratio
                evaluation_per_score_and_outcome[outcome]["scores"][score]["o:e ratio 95%CI"] = [float(oe_ratio_ci_lower), float(oe_ratio_ci_upper)]
                evaluation_per_score_and_outcome[outcome]["scores"][score]["o:e ratio p-value"] = oe_ratio_pval
                evaluation_per_score_and_outcome[outcome]["scores"][score]["ici"] = ici
                evaluation_per_score_and_outcome[outcome]["scores"][score]["e50"] = e50
                evaluation_per_score_and_outcome[outcome]["scores"][score]["e90"] = e90
                evaluation_per_score_and_outcome[outcome]["scores"][score]["emax"] = emax                
                
                # reclassification
                if score != "CardioExplorer":
                    if "CardioExplorer" in scores:
                        score_key = "CardioExplorer"
                    else:
                        score_key = "CardioExplorer individual PTP"
                    df_reclassification = pd.DataFrame({"y_true": y_true, "score": y_pred_prob, "CardioExplorer": scores[score_key]["ptp_values"]})
                    continuos_nri_df, _ = nri(df_reclassification, "score", "CardioExplorer", "y_true")
                    nri_threshold_df = pd.DataFrame()
                    for threshold in bins[1:-1]:
                        nri_threshold_df = pd.concat([nri_threshold_df, calculate_nri_with_threshold(df_reclassification, "score", "CardioExplorer", "y_true", threshold)]) 
                    cat1 = pd.cut(y_pred_prob, bins=bins, labels=guideline_thresholds)
                    cat2 = pd.cut(scores[score_key]["ptp_values"], bins=bins, labels=guideline_thresholds)
                    df_cook_table = pd.DataFrame({'true': y_true, score: cat1, 'CardioExplorer': cat2})
                    table_event = pd.crosstab(df_cook_table[df_cook_table['true'] == 1][score], df_cook_table[df_cook_table['true'] == 1]['CardioExplorer'])
                    table_event = table_event.applymap(lambda n : f"{n} ({round(n/table_event.sum().sum(), 2)})")
                    table_nonevent = pd.crosstab(df_cook_table[df_cook_table['true'] == 0][score], df_cook_table[df_cook_table['true'] == 0]['CardioExplorer'])
                    table_nonevent = table_nonevent.applymap(lambda n : f"{n} ({round(n/table_nonevent.sum().sum(), 2)})")
                    idi = evaluation_per_score_and_outcome[outcome]["scores"][score_key]["discrimination slope"] - discrimination_slope
                    categorial_nri, categorial_nri_ci, categorial_nri_p = bootstrap_nri(y_pred_prob, scores[score_key]["ptp_values"], y_true, bins)
        
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["CardioExplorer continous nri"] = continuos_nri_df # probably too optimistic
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["CardioExplorer Cook's reclassification table event"] = table_event
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["CardioExplorer Cook's reclassification table nonevent"] = table_nonevent
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["CardioExplorer Integrated Discrimination Improvement"] = continuos_nri_df.loc["IDI", :]
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["CardioExplorer categorial nri"] = pd.DataFrame({"NRI": {"index with 95 CI": f"{round(categorial_nri, 2)}({round(categorial_nri_ci[0], 2)}-{round(categorial_nri_ci[1], 2)})", "p-value": categorial_nri_p}}).T
                    evaluation_per_score_and_outcome[outcome]["scores"][score]["CardioExplorer threshold-based nri"] = nri_threshold_df # probably too optimistic
                
                # decision curve analysis
                dca_df = calculate_dca(y_true, y_pred_prob, model_name=score, harm=0)
                evaluation_per_score_and_outcome[outcome]["scores"][score]["dca_df"] = dca_df
                
                dca_df_harm = calculate_dca(y_true, y_pred_prob, model_name=score, harm=harms_per_score[score])
                evaluation_per_score_and_outcome[outcome]["scores"][score]["dca_df_including_test_harm"] = dca_df_harm
                evaluation_per_score_and_outcome[outcome]["scores"][score]["test_harm"] = harms_per_score[score]
                
    score_comparison_scores_and_outcomes[score_comparison_key]["evaluation_per_score_and_outcome"] = evaluation_per_score_and_outcome

## Create folder structure for analysis

In [ ]:
output_folder = "analysis"
folders_to_create = ["general", "calibration", "discrimination", "clinical_usefulness", "reclassification"]
output_folder = Path(output_folder)
now = datetime.now().strftime("%Y-%m-%d_%H-%M")
outpath = output_folder.joinpath(now)
Path.mkdir(outpath)
for folder in folders_to_create:
    Path.mkdir(outpath.joinpath(folder))

In [ ]:
def multi_replace(text: str, replacements: dict) -> str:
    pattern = re.compile("|".join(map(re.escape, replacements.keys())))
    return pattern.sub(lambda m: replacements[m.group(0)], text)

short_scores = {"CardioExplorer individual PTP": "CEi", "CardioExplorer": "CE", "PROMISE Minimal-Risk Score": "PMRS", "ESC 2024 PTP (formula)": "E24PTP", "ESC 2024 RF-CL (formula)": "E24RFCL", "ESC 2019 PTP": "E19PTP", "ESC 2013 PTP": "E13PTP", "ESC 2024 CACS-CL (formula)": "E24CSCL", "ACC 2021 PTP": "A21PTP", "ACC 2012 PTP": "A12PTP", 'Diamond-Forrester': "DF", "ESC 2024 RF-CL (formula with default 0 for number of risk factors and symptom score)": "E24RFCLd", "CAD consortium score 2012 Basic PTP": "CAD12PTP", "CAD consortium score 2012 Clinical PTP": "CAD12CL", "clinical assessment of CAD pretest probability": "clin"}

## Plots start from here:

In [ ]:
per_group_variables.to_excel(outpath.joinpath("general/per_group_variables.xlsx"))
per_group_variables

In [ ]:
against_reference_variables.to_excel(outpath.joinpath("general/reference.xlsx"))
against_reference_variables

In [ ]:
against_reference_variables_z.to_excel(outpath.joinpath("general/reference_z.xlsx"))
against_reference_variables_z

In [ ]:
for score_comparison_key in score_comparison_scores_and_outcomes:
    short_key = multi_replace(score_comparison_key, short_scores)
    short_key = short_key.replace(" vs. ", "-")
    Path.mkdir(outpath.joinpath(f"calibration/{short_key}"))
    for outcome, outcome_dict in score_comparison_scores_and_outcomes[score_comparison_key]["outcomes"].items():
        Path.mkdir(outpath.joinpath(f"calibration/{short_key}/{outcome}"))
        evaluation_per_score_and_outcome = score_comparison_scores_and_outcomes[score_comparison_key]["evaluation_per_score_and_outcome"]
        y_true = outcome_dict["y_true"]
        scores = outcome_dict["scores"]
        combined_calibration_plot = {}
        for score, score_dict in scores.items(): 
            if all([np.count_nonzero(y_true == y) > 1 for y in np.unique(y_true)]) and len(np.unique(y_true)) == 2:
                ce = evaluation_per_score_and_outcome[outcome]["scores"][score]["ce"]
                # nonparametric is a calibration curve derived using the LOWESS algorithm
                plot, x_nonparametric, y_nonparametric = ce.calibration_plot(colors={"Perfectly calibrated": "black", "Nonparametric": colors_hex[score], "histogram": "grey", "Grouped observations": "red"}, frac=0.2)
                if isinstance(evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_intercept"], str):
                    plt.title(f"Calibration Intercept: {evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_intercept"]}, Calibration Slope: {evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_slope"]}\nn={len(y_true)}", fontsize=10)
                else:
                    plt.title(f"Calibration Intercept: {evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_intercept"]:.4f}, Calibration Slope: {evaluation_per_score_and_outcome[outcome]["scores"][score]["calibration_slope"]:.4f}\nn={len(y_true)}", fontsize=10)
                plt.suptitle(f"Calibration plot for {score} compared to {outcome}", fontsize=14)
                plt.show(plot)
                plot.savefig(outpath.joinpath(f"calibration/{short_key}/{outcome}/calibration_plot_{score}.svg"))
                plot.savefig(outpath.joinpath(f"calibration/{short_key}/{outcome}/calibration_plot_{score}.png"))
                combined_calibration_plot[score] = (x_nonparametric, y_nonparametric)
                
                predictions = scores[score]["ptp_values"]
                cases = predictions[y_true == 1]
                non_cases = predictions[y_true == 0]

                fig, ax = plt.subplots()
                sns.histplot(non_cases, bins=30, alpha=0.6, label='non-cases',
                    color='blue', kde=True, stat='count', ax=ax)
                sns.histplot(-cases, bins=30, alpha=0.6, label='cases',
                    color='yellow', kde=True, stat='count', ax=ax)
                plt.ylim(-len(scores[score]["ptp_values"]), len(scores[score]["ptp_values"]))
                plt.xlim(-1, 1)
                plt.xlabel("Predicted probability")
                plt.title(f"Distribution of predictions for {score}")
                plt.show()
                fig.savefig(outpath.joinpath(f"calibration/{short_key}/{outcome}/histogram_mirror_{score}.svg"))
                fig.savefig(outpath.joinpath(f"calibration/{short_key}/{outcome}/histogram_mirror_{score}.png"))

                fig, ax = plt.subplots()
                sns.histplot(data=scores[score]["ptp_values"])
                plt.ylim(-len(scores[score]["ptp_values"]), len(scores[score]["ptp_values"]))
                plt.xlim(0, 1)
                plt.xlabel("Predicted probability")
                plt.title(f"Distribution of predictions for {score}")
                plt.show()
                fig.savefig(outpath.joinpath(f"calibration/{short_key}/{outcome}/histogram_combined_{score}.svg"))
                fig.savefig(outpath.joinpath(f"calibration/{short_key}/{outcome}/histogram_combined_{score}.png"))
        
        fig, ax = plt.subplots(figsize=(8, 8))
        for score_name, xy in combined_calibration_plot.items():
            x, y = xy[0], xy[1]
            ax.plot(x, y, label=score_name, color=colors_hex[score_name])
        ax.set_xlabel('Predicted Probability')
        ax.set_ylabel('Actual Probability')
        ax.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
        plt.suptitle(f"Combined calibration plot compared to {outcome}", fontsize=14)
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.legend()
        plt.show()
        fig.savefig(outpath.joinpath(f"calibration/{short_key}/{outcome}/combined_calibration_plot.svg"))
        fig.savefig(outpath.joinpath(f"calibration/{short_key}/{outcome}/combined_calibration_plot.png"))

In [ ]:
# diagnostic accuracy (AUC plots)
for score_comparison_key in score_comparison_scores_and_outcomes:
    short_key = multi_replace(score_comparison_key, short_scores)
    short_key = short_key.replace(" vs. ", "-")
    Path.mkdir(outpath.joinpath(f"discrimination/{short_key}/"))
    for outcome, outcome_dict in score_comparison_scores_and_outcomes[score_comparison_key]["outcomes"].items():
        evaluation_per_score_and_outcome = score_comparison_scores_and_outcomes[score_comparison_key]["evaluation_per_score_and_outcome"]
        if outcome in evaluation_per_score_and_outcome:
            Path.mkdir(outpath.joinpath(f"discrimination/{short_key}/{outcome}/"))
            y_true = outcome_dict["y_true"]
            scores = outcome_dict["scores"]
            fig, ax = plt.subplots()
            plot_chance_level = True
            for score, score_dict in scores.items(): 
                score_values = score_dict["score_values"]
                metrics.RocCurveDisplay.from_predictions(
                    y_true,
                    score_values,
                    name=score,
                    plot_chance_level=plot_chance_level,
                    despine=True,
                    ax=ax,
                    color=colors_hex[score],
                )
                plot_chance_level = False
            _ = ax.set(
                xlabel="False Positive Rate",
                ylabel="True Positive Rate",
            )
            plt.suptitle(f"Diagnostic accuracy compared to {outcome}", fontsize=14)
            plt.title(f"n={len(outcome_dict['y_true'])}", fontsize=10)
            plt.show()
            fig.savefig(outpath.joinpath(f"discrimination/{short_key}/{outcome}/comparison_AUC.svg"))
            fig.savefig(outpath.joinpath(f"discrimination/{short_key}/{outcome}/comparison_AUC.png"))
    
            score_color_dict_density_plot = {}
            for score, score_dict in scores.items():
                color = colors_hex[score]
                color_neg = adjust_color_brightness(color, 0.5)
                color_pos = adjust_color_brightness(color, 1.5)
                score_color_dict_density_plot[f"{score} positive"] = color_pos
                score_color_dict_density_plot[f"{score} negative"] = color_neg
                score_values = score_dict["score_values"]
                discrimination_slope = evaluation_per_score_and_outcome[outcome]["scores"][score]["discrimination slope"]
                df_pred = pd.DataFrame({'y_true': y_true, 'score_values': score_values})
                df_pred['y_true_label'] = df_pred['y_true'].map({0: 'negative', 1: 'positive'})
                fig = plt.figure(figsize=(8, 5))
                sns.boxplot(data=df_pred, x='y_true_label', y='score_values', palette={"negative": color_neg, "positive": color_pos})
                plt.suptitle(f"Discrimination box plot for {score} compared to {outcome}", fontsize=14)
                plt.title(f"Discrimination Slope: {discrimination_slope:.3f}\nn={len(outcome_dict['y_true'])}", fontsize=10)
                plt.ylabel("Predicted probability")
                plt.xlabel("Outcome")
                plt.grid(True, axis='y', linestyle='--', alpha=0.5)
                plt.tight_layout()
                plt.show()
                fig.savefig(outpath.joinpath(f"discrimination/{short_key}/{outcome}/{short_scores[score]}_boxplot.svg"))
                fig.savefig(outpath.joinpath(f"discrimination/{short_key}/{outcome}/{short_scores[score]}_boxplot.png"))
        
                fig = plt.figure(figsize=(8, 5))
                sns.kdeplot(data=df_pred, x='score_values', hue='y_true_label', fill=True, common_norm=False, palette={"negative": color_neg, "positive": color_pos}, alpha=0.15)
                mean_neg = df_pred.loc[df_pred["y_true"] == 0, "score_values"].mean()
                mean_pos = df_pred.loc[df_pred["y_true"] == 1, "score_values"].mean()
                plt.axvline(mean_neg, color=color_neg, linestyle='--', label=f'Mean negative: {mean_neg:.2f}')
                plt.axvline(mean_pos, color=color_pos, linestyle='--', label=f'Mean positive: {mean_pos:.2f}')
                plt.suptitle(f"Discrimination density plot for {score} compared to {outcome}", fontsize=14)
                plt.title(f"Discrimination Slope: {discrimination_slope:.3f}\nn={len(outcome_dict['y_true'])}", fontsize=10)
                plt.ylabel("Density")
                plt.xlabel("Score value")
                plt.legend()
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.tight_layout()
                plt.show()
                fig.savefig(outpath.joinpath(f"discrimination/{short_key}/{outcome}/{short_scores[score]}_kdeplot.svg"))
                fig.savefig(outpath.joinpath(f"discrimination/{short_key}/{outcome}/{short_scores[score]}_kdeplot.png"))
        
            df_preds = pd.DataFrame()
            for score, score_dict in scores.items():
                score_values = score_dict["score_values"]
                scaler = MinMaxScaler()
                score_values = scaler.fit_transform(score_values.reshape(-1, 1)).flatten()
                df_pred = pd.DataFrame({'score_values': score_values, 'y_true': y_true, 'score': score})
                df_preds = pd.concat([df_preds, df_pred], ignore_index=True)
            df_preds['score outcome'] = df_preds['score'] + " " + df_preds['y_true'].replace({0: "negative", 1: "positive"}).astype(str)
        
            fig = plt.figure(figsize=(10, 6))
            sns.kdeplot(data=df_preds, x='score_values', hue='score outcome', multiple='layer', fill=True, alpha=0.3, linewidth=1.5, palette=score_color_dict_density_plot)
            plt.suptitle(f"Discrimination density plot compared to {outcome}", fontsize=14)
            plt.title(f"n={len(outcome_dict['y_true'])}", fontsize=10)
            plt.xlabel("Score value normalised")
            plt.ylabel("Density")
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.tight_layout()
            plt.show()
            fig.savefig(outpath.joinpath(f"discrimination/{short_key}/{outcome}/comparison_kdeplot.svg"))
            fig.savefig(outpath.joinpath(f"discrimination/{short_key}/{outcome}/comparison_kdeplot.png"))
        
            specificity_and_sensitivity_per_score_and_threshold = {}
            for i in scores:
                entries = []
                for j in ["sensitivity", "specificity", "ppv", "npv"]:
                    for k in [0.05, 0.15, 0.85]:
                        entries.append(((j, k), evaluation_per_score_and_outcome[outcome]["scores"][i][f"{j} at {k: .0%} threshold"] ))
                specificity_and_sensitivity_per_score_and_threshold[i] = dict(entries)
            specificity_and_sensitivity_per_score_and_threshold_df = pd.DataFrame(specificity_and_sensitivity_per_score_and_threshold)
            specificity_and_sensitivity_per_score_and_threshold_df.to_excel(outpath.joinpath(f"discrimination/{short_key}/{outcome}/specificity_sensitivity.xlsx"))
            print("specificity and sensitivity per score and threshold for", outcome)
            display(specificity_and_sensitivity_per_score_and_threshold_df)

In [ ]:
# reclassification
for score_comparison_key in score_comparison_scores_and_outcomes:
    short_key = multi_replace(score_comparison_key, short_scores)
    short_key = short_key.replace(" vs. ", "-")
    Path.mkdir(outpath.joinpath(f"reclassification/{short_key}/"))
    for outcome, outcome_dict in score_comparison_scores_and_outcomes[score_comparison_key]["outcomes"].items():
        evaluation_per_score_and_outcome = score_comparison_scores_and_outcomes[score_comparison_key]["evaluation_per_score_and_outcome"]
        if outcome in evaluation_per_score_and_outcome:
            Path.mkdir(outpath.joinpath(f"reclassification/{short_key}/{outcome}/"))
            for score, score_dict in evaluation_per_score_and_outcome[outcome]["scores"].items():
                if score != "CardioExplorer":
                    print("Cook's reclassification table for events")
                    display(score_dict["CardioExplorer Cook's reclassification table event"])
                    print("Cook's reclassification table for non-events")
                    display(score_dict["CardioExplorer Cook's reclassification table nonevent"])
                    score_dict["CardioExplorer Cook's reclassification table event"].to_excel(outpath.joinpath(f"reclassification/{short_key}/{outcome}/Cook table events CE vs. {short_scores[score]}.xlsx"))
                    score_dict["CardioExplorer Cook's reclassification table nonevent"].to_excel(outpath.joinpath(f"reclassification/{short_key}/{outcome}/Cook table nonevents CE vs. {short_scores[score]}.xlsx"))

In [ ]:
# DCA plot 
show_standardized = False
# plots adapted from https://mskcc-epi-bio.github.io/decisioncurveanalysis/dca-tutorial-python.html
for score_comparison_key in score_comparison_scores_and_outcomes:
    short_key = multi_replace(score_comparison_key, short_scores)
    short_key = short_key.replace(" vs. ", "-")
    Path.mkdir(outpath.joinpath(f"clinical_usefulness/{short_key}/"))
    for outcome, outcome_dict in score_comparison_scores_and_outcomes[score_comparison_key]["outcomes"].items():
        evaluation_per_score_and_outcome = score_comparison_scores_and_outcomes[score_comparison_key]["evaluation_per_score_and_outcome"]
        if outcome in evaluation_per_score_and_outcome:
            Path.mkdir(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/"))
            scores = outcome_dict["scores"]
            y_true = outcome_dict["y_true"]
            prevalence = np.mean(y_true)
            fig, ax = plt.subplots()
            for score, score_dict in evaluation_per_score_and_outcome[outcome]["scores"].items():
                plot_dca(score_dict["dca_df"], ax, score_dict["dca_df"]["standardized_net_benefit"] if show_standardized else score_dict["dca_df"]["net_benefit"], score, smooth_fraction=0.5, color=colors_hex[score])
            for strategy, strategy_dict in evaluation_per_score_and_outcome[outcome]["extreme_strategies"].items():
                plot_dca(strategy_dict["dca_df"], ax, strategy_dict["dca_df"]["standardized_net_benefit"] if show_standardized else strategy_dict["dca_df"]["net_benefit"], strategy, smooth_fraction=0.5, color=colors_hex[strategy])
            plt.suptitle(f"Decision Curve Analysis for {outcome}", fontsize=14)
            plt.title(f"n={len(outcome_dict["y_true"])}", fontsize=10)
            plt.xlabel("Threshold Probability")
            plt.ylabel("Standardized Net Benefit" if show_standardized else "Net Benefit")
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.ylim(-0.05, 1.1)
            plt.xlim(0, 0.3) # plots only the area with relevant clinical decision thresholds
            plt.show()
            fig.savefig(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/DCA_plot.svg"))
            fig.savefig(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/DCA_plot.png"))

            fig, ax = plt.subplots()
            for score, score_dict in evaluation_per_score_and_outcome[outcome]["scores"].items():
                plot_dca(score_dict["dca_df"], ax, score_dict["dca_df"]["standardized_net_benefit_negative"] if show_standardized else score_dict["dca_df"]["net_benefit_negative"], score, smooth_fraction=0.5, color=colors_hex[score])
            for strategy, strategy_dict in evaluation_per_score_and_outcome[outcome]["extreme_strategies"].items():
                plot_dca(strategy_dict["dca_df"], ax, strategy_dict["dca_df"]["standardized_net_benefit_negative"] if show_standardized else strategy_dict["dca_df"]["net_benefit_negative"], strategy, smooth_fraction=0.5, color=colors_hex[strategy])
            plt.suptitle(f"Decision Curve Analysis for {outcome}", fontsize=14)
            plt.title(f"n = {len(outcome_dict["y_true"])}\n NB negative", fontsize=10)
            plt.xlabel("Threshold Probability")
            plt.ylabel("Standardized Net Benefit Negative" if show_standardized else "Net Benefit Negative")
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.ylim(-0.05, 1.1)
            plt.xlim(0, 0.3) # plots only the area with relevant clinical decision thresholds
            plt.show()
            fig.savefig(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/DCA_negative_plot.svg"))
            fig.savefig(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/DCA_negative_plot.png"))
            
            fig, ax = plt.subplots()
            for score, score_dict in evaluation_per_score_and_outcome[outcome]["scores"].items():
                overall_benefit = (score_dict["dca_df"]["standardized_net_benefit"] + score_dict["dca_df"]["standardized_net_benefit_negative"]) if show_standardized else (score_dict["dca_df"]["net_benefit"] + score_dict["dca_df"]["net_benefit_negative"])
                plot_dca(score_dict["dca_df"], ax, overall_benefit, score, smooth_fraction=0.5, color=colors_hex[score])
            for strategy, strategy_dict in evaluation_per_score_and_outcome[outcome]["extreme_strategies"].items():
                overall_benefit = (strategy_dict["dca_df"]["standardized_net_benefit_negative"] + strategy_dict["dca_df"]["standardized_net_benefit"]) if show_standardized else (strategy_dict["dca_df"]["net_benefit_negative"] + strategy_dict["dca_df"]["net_benefit"])
                plot_dca(strategy_dict["dca_df"], ax, overall_benefit, strategy, smooth_fraction=0.5, color=colors_hex[strategy])
            plt.suptitle(f"Decision Curve Analysis for {outcome}", fontsize=14)
            plt.title(f"n = {len(outcome_dict["y_true"])}\n Overall Benefit", fontsize=10)
            plt.xlabel("Threshold Probability")
            plt.ylabel("Standardized Overall Benefit" if show_standardized else "Overall Benefit")
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.ylim(-0.05, 1.1)
            plt.xlim(0, 0.3) # plots only the area with relevant clinical decision thresholds
            plt.show()
            fig.savefig(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/DCA_overall_plot.svg"))
            fig.savefig(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/DCA_overall_plot.png"))

In [ ]:
CAD_relevant_diagnostic = []
not_CAD_relevant_diagnostic = []

In [ ]:
colors = plt.get_cmap("tab20b").colors
diagnostics_colors = {
    key: f"rgb({int(r*255)}, {int(g*255)}, {int(b*255)})"
    for i, (key, (r, g, b)) in enumerate(
        zip(CAD_relevant_diagnostic, [colors[i % len(colors)] for i in range(len(CAD_relevant_diagnostic))]))
}

In [ ]:
threshold_for_pie = 0.03
for group, pseudonyms in pseudonyms_per_group.items():
    diagnostic_pseudonym = {}
    multiple_diagnostics_n = 0
    category_count = {k.lower(): 0 for k in CAD_relevant_diagnostic}
    for i, row in df_data.iterrows():
        pseudonym = row["pseudonym"]
        if pseudonym in pseudonyms:
            diagnostics = row["performed, planned, or recommended initial diagnostics"].split(", ")
            diagnostics = [d.replace("-", " ") for d in diagnostics]
            diagnostic_pseudonym[pseudonym] = {"diagnostic": diagnostics}
            for d in diagnostics:
                matching_category = [c for c in category_count if c in d.lower()]
                if matching_category:
                    category_count[matching_category[0]]+=1
                else:
                    if d not in not_CAD_relevant_diagnostic:
                        print(d)
            multiple_diagnostics_n += len(diagnostics)
    category_count_for_pie = {"Others": 0}
    for c, v in category_count.items():
        if c == "Others":
            category_count_for_pie[c] += v
        else:
            if (v/multiple_diagnostics_n) > threshold_for_pie:
                category_count_for_pie[c] = v
            else:
                category_count_for_pie["Others"] += v
    fig = go.Figure(data=[go.Pie(labels=list(category_count_for_pie.keys()), values=list(category_count_for_pie.values()), title=f"Diagnostics for cohort {group} n={len(pseudonyms)}, n(multiple categories)={multiple_diagnostics_n}", marker=dict(colors=[diagnostics_colors[k] for k in category_count_for_pie.keys()]), hole=0)])
    fig.show()
    fig.write_image(outpath.joinpath(f"general/{group}_recommended_diagnostics_pie_chart.png"), engine="orca")
    fig.write_image(outpath.joinpath(f"general/{group}_recommended_diagnostics_pie_chart.svg"), engine="orca")

In [ ]:
diagnosis_to_category = {
    "Hypertophic cardiomyopathy": [],
    "Acute myocardial infarction": [],
    "Heart failure": [],
    "Coronary artery disease": [],
    "Other chest pain": [],
    "Atrial fibrillation": [],
    "Dysrhythmias other than atrial fibrillation": [], 
    "Myocarditis": [],
    "Valvular disease": [],
    "Hypertension": [],

    "Others": [],
    "follow-up": [],
    
    "Pericardial effusion": [],
    "Connatal heart defect": [], 

    "Aortic dissection": [],
    "Aneurysm": [],
    "Aortic stenosis": [],
    

    "Pneumonia": [],
    "Pulmonary embolism": [],
    "Pneumothorax": [],
    "Pleuritis": [],
    "Pleural effusion": [],
    "COPD": [],

    "Abdominal causes": [],


}

In [ ]:
colors = plt.get_cmap("tab20b").colors
diagnosis_colors = {
    key: f"rgb({int(r*255)}, {int(g*255)}, {int(b*255)})"
    for i, (key, (r, g, b)) in enumerate(
        zip(diagnosis_to_category.keys(), [colors[i % len(colors)] for i in range(len(diagnosis_to_category))])
    )
}

In [ ]:
threshold_for_pie = 0.03
for group, pseudonyms in pseudonyms_per_group.items():
    diagnosis_category_pseudonym = {}
    category_count = {k: 0 for k in diagnosis_to_category.keys()}
    multiple_categories_n = 0
    for i, row in df_data.iterrows():
        pseudonym = row["pseudonym"]
        if pseudonym in pseudonyms:
            diagnosis = row["diagnosis at discharge or inpatient admission"]
            category = [c for c, examples in diagnosis_to_category.items() if any([str(e).lower() in str(diagnosis).lower() for e in examples])] 
            diagnosis_category_pseudonym[pseudonym] = {"diagnosis": diagnosis, "category": category, "multiple_categories": len(category)}
            for c in category:
                category_count[c] += 1
            if not category:
                print(diagnosis)
            multiple_categories_n += len(category)
    category_count_for_pie = {"Others": 0}
    for c, v in category_count.items():
        if c == "Others":
            category_count_for_pie[c] += v
        else:
            if (v/multiple_categories_n) > threshold_for_pie:
                category_count_for_pie[c] = v
            else:
                print(c, v)
                category_count_for_pie["Others"] += v
    fig = go.Figure(data=[go.Pie(labels=list(category_count_for_pie.keys()), values=list(category_count_for_pie.values()), title=f"Diagnosis categorized for cohort {group} n={len(pseudonyms)}, n(multiple categories)={multiple_categories_n}", marker=dict(colors=[diagnosis_colors[k] for k in category_count_for_pie.keys()]), hole=0)])
    fig.show()
    fig.write_image(outpath.joinpath(f"general/{group}_included_diagnoses_pie_chart.png"), engine="orca")
    fig.write_image(outpath.joinpath(f"general/{group}_included_diagnoses_pie_chart.svg"), engine="orca")

In [ ]:
keys_for_df = ["brier", "nk_r2", "cs_r2", 'sensitivity at  5% threshold', 'specificity at  5% threshold', 'sensitivity at  15% threshold', 'specificity at  15% threshold', 'sensitivity at  85% threshold', 'specificity at  85% threshold', 'ppv at  5% threshold', 'npv at  5% threshold', 'ppv at  15% threshold', 'npv at  15% threshold', 'ppv at  85% threshold', 'npv at  85% threshold', 'auc', 'discrimination slope', 'hosmerlemeshow_statistic', 'hosmerlemeshow_p', 'adaptive calibration error', 'maximum calibration error', 'calibration_intercept', "calibration_intercept_ci", 'calibration_slope', "calibration_slope_ci", "o:e ratio", "o:e ratio 95%CI", "o:e ratio p-value", "ici", "e50", "e90"]
for score_comparison_key in score_comparison_scores_and_outcomes:
    short_key = multi_replace(score_comparison_key, short_scores)
    short_key = short_key.replace(" vs. ", "-")
    Path.mkdir(outpath.joinpath(f"general/{short_key}"))
    for outcome, outcome_dict in score_comparison_scores_and_outcomes[score_comparison_key]["outcomes"].items():
        Path.mkdir(outpath.joinpath(f"general/{short_key}/{outcome}/"))
        evaluation_per_score_and_outcome = score_comparison_scores_and_outcomes[score_comparison_key]["evaluation_per_score_and_outcome"]
        scores = outcome_dict["scores"]
        expected_observed_categories = pd.DataFrame()
        for score, score_dict in scores.items(): 
            if outcome in evaluation_per_score_and_outcome:
                s = evaluation_per_score_and_outcome[outcome]["scores"][score]
                pd.Series({key: s[key] for key in keys_for_df}).to_excel(outpath.joinpath(f"general/{short_key}/{outcome}/{score}_metrices.xlsx"))
                s['expected vs. observed outcome ESC 2024 guideline categories'].name = "true prevalence in categories predicted by " + short_scores[score]
                expected_observed_categories = pd.concat([expected_observed_categories, s['expected vs. observed outcome ESC 2024 guideline categories']], axis=1)
                s["dca_df"].to_excel(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/{short_scores[score]}_dca.xlsx"))
                s['dca_df_including_test_harm']["test harm"] = s['test_harm']
                s['dca_df_including_test_harm'].to_excel(outpath.joinpath(f"clinical_usefulness/{short_key}/{outcome}/{short_scores[score]}_dca_including_harm.xlsx"))
                if not isinstance(s['ce_contingency_table'], float):
                    s['ce_contingency_table'].to_excel(outpath.joinpath(f"calibration/{short_key}/{outcome}/{short_scores[score]}_contingency_table.xlsx"))
        expected_observed_categories.to_excel(outpath.joinpath(f"general/{short_key}/{outcome}/comparison_expected vs. observed outcome.xlsx"))

In [ ]:
assessor_list = ["MACE 3 months (Major Adverse Cardiovascular Events)", "percutaneous coronary angiography performed 3 months", "non-invasive ischemia testing performed 3 months", 
                     "coronary computed tomography angiography performed 3 months", "stress electrocardiography performed 3 months"]
for score in ['CardioExplorer',
 "CardioExplorer individual PTP",
 'PROMISE Minimal-Risk Score', 
 'ESC 2024 PTP (formula)', 
 'ESC 2024 RF-CL (formula)', 
 'ESC 2019 PTP',
 'ACC 2021 PTP',
 'Diamond-Forrester',
 'ESC 2024 RF-CL (formula with default 0 for number of risk factors and symptom score)',
 'CAD consortium score 2012 Basic PTP',
 'CAD consortium score 2012 Clinical PTP']:
    bins = [0.0, 0.05, 0.15, 0.5, 0.85, 1.0]
    guideline_thresholds = ["<5%", '5–15%', '15–50%', '50–85%', '>85%']
    score_subset = df_scores.loc[:, [(score, "ptp_values"), ("pseudonym", " ")]]
    score_subset = score_subset.loc[~score_subset[(score, "ptp_values")].isna(), :]
    score_subset = pd.DataFrame({
        'pseudonym': score_subset[("pseudonym", " ")], 
        f'category ({score})': pd.cut(score_subset[(score, "ptp_values")], bins, labels=guideline_thresholds, include_lowest=True)
                                })
    for assessor in assessor_list:
        score_subset = score_subset.merge(df_data[['pseudonym', assessor]], on='pseudonym', how='left')
    score_subset.dropna(inplace=True)
    score_subset = score_subset.replace({"no information": 0, "none": 0, "no": 0})
    score_subset["no further diagnosis"] = (~(score_subset[assessor_list]!=0).any(axis=1)).astype(int)
    score_subset[assessor_list] = (score_subset[assessor_list] != 0).astype(int)
    full_assessor_list = assessor_list + ["no further diagnosis"]
    score_subset = score_subset.groupby(f'category ({score})')[full_assessor_list].sum()
    score_subset.to_excel(outpath.joinpath(f"general/diagnostics_per_category_{score}.xlsx"))